In [ ]:
# Clone repository
import os, sys

import yaml

os.chdir("/content")
if not os.path.isdir("RL_DEMO"):
  !git clone https://github.com/Kyu3224/RL_DEMO.git
else:
  print("Cloned Directory already exists")

os.chdir("/content/RL_DEMO")
print("Current Directory: ", os.getcwd())

sys.path.insert(0, "/content/RL_DEMO")
os.environ["MUJOCO_GL"] = "egl"

In [ ]:
# Install dependencies
!pip install torch numpy tensorboard gymnasium==0.29.1 stable-baselines3==2.3.0 mujoco==3.1.5 imageio

In [ ]:
import numpy as np
import importlib

import os
import gc
import time
import imageio
import torch

from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import EvalCallback, CheckpointCallback, CallbackList
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import SubprocVecEnv
from IPython.display import Video, display
from pathlib import Path

import src.go1_mujoco_env as go1_env

from src.utils.reward_logging_callback import RewardLoggingCallback

DEFAULT_CAMERA_CONFIG = {
    "azimuth": 90.0,
    "distance": 3.0,
    "elevation": -25.0,
    "lookat": np.array([0., 0., 0.]),
    "fixedcamid": 0,
    "trackbodyid": -1,
    "type": 2,
}

policy_cfg_path = Path("/content/RL_DEMO/src/params.yaml")
with policy_cfg_path.open("r", encoding="utf-8") as f:
    policy_cfg = yaml.safe_load(f)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/RL_DEMO/logs

In [ ]:
importlib.reload(go1_env)

# Train
MODEL_DIR = "models"
LOG_DIR = "logs"

os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)

vec_env = make_vec_env(
    go1_env.Go1MujocoEnv,
    env_kwargs={"prj_path": "/content/RL_DEMO"},
    n_envs=policy_cfg["n_envs"],
    seed=policy_cfg["seed"],
    vec_env_cls=SubprocVecEnv,
)

train_time = time.strftime("%Y-%m-%d_%H-%M-%S")
run_name = f"{train_time}"

model_path = f"{MODEL_DIR}/{run_name}"
print(
    f"Training on {policy_cfg['n_envs']} parallel training environments and saving models to '{model_path}'"
)

checkpoint_callback = CheckpointCallback(
    save_freq=policy_cfg["policy"]["n_steps"] * policy_cfg["log"]["interval"],  # e.g. 100_000
    save_path=model_path,  # directory
    name_prefix="model",  # checkpoint_model_100000_steps.zip
    save_replay_buffer=False,
    save_vecnormalize=False,
)

eval_callback = EvalCallback(
    vec_env,
    best_model_save_path=model_path,
    log_path=LOG_DIR,
    eval_freq=policy_cfg["eval_freq"],
    n_eval_episodes=5,
    deterministic=True,
    render=False,
)

reward_logging_callback = RewardLoggingCallback()

callbacks = CallbackList([
    eval_callback,
    checkpoint_callback,
    reward_logging_callback,
])

PRETRAINED_MODEL_PATH = "/content/RL_DEMO/src/models/pretrained/final_model.zip"

print(f"Using Pretrained model from {PRETRAINED_MODEL_PATH}")

model = PPO.load(path=PRETRAINED_MODEL_PATH, env=vec_env)

model.tensorboard_log = LOG_DIR

model.learn(
    total_timesteps=policy_cfg["total_timestep"],
    reset_num_timesteps=True,
    progress_bar=True,
    tb_log_name=run_name,
    callback=callbacks,
)
# Save final model
model.save(f"{model_path}/final_model")

vec_env.close()

del model
del eval_callback
del vec_env

gc.collect()

In [ ]:
# Test
importlib.reload(go1_env)
model_path = "/content/RL_DEMO/src/models/pretrained/final_model"

WIDTH, HEIGHT = 544, 368

env = go1_env.Go1MujocoEnv(
    prj_path="/content/RL_DEMO",
    render_mode="rgb_array",
    camera_name="tracking",
    width=WIDTH,
    height=HEIGHT,
)
inter_frame_sleep = 0.0

model = PPO.load(path=model_path, env=env, verbose=1)

video_path = "/content/rollout.mp4"

frames = []

obs, _ = env.reset()
max_time_step_s = policy_cfg["test"]["max_time_step_s"]
ep_len = 0
video_fps = 25

# Ctrl Hz: 50
render_interval = 50 // video_fps
max_steps = int(max_time_step_s * 50)

writer = imageio.get_writer(
    video_path,
    fps=video_fps,
    codec="libx264",
    quality=8,
    pixelformat="yuv420p",
)

frames = []

while ep_len < max_steps:
  with torch.no_grad():
    action, _ = model.predict(obs, deterministic=True)
  obs, reward, terminated, truncated, info = env.step(action)

  if ep_len % render_interval == 0:
    frame = env.render()
    frames.append(frame)

  ep_len += 1

imageio.mimwrite(
    video_path,
    frames,
    fps=video_fps,
    codec="libx264",
    quality=8,
    pixelformat="yuv420p",
)

env.close()

print("Saved video to:", video_path)

display(
    Video(
        video_path,
        embed=True,
        html_attributes="controls autoplay loop"
    )
)